<h1> Project Progress </h1>

This file will track the progress of this project, including major steps, code changes, results, etc. 
A ipynb file is best as it will allow for code, writing, etc and best formatting. 

<h1> Baseline: Full Observability: Catch only <h1>

In [ ]:
#Observations

def getAgentObs(self,agentIndex):
    W, H = self.grid.width, self.grid.height
    ax, ay = self.agents[agentIndex].x, self.agents[agentIndex].y
    gx, gy = self.ghostCoords

    numAgents = len(self.agents)
    obs = [
        ax / (W-1), ay / (H-1),
        gx / (W-1), gy / (H-1),
        self.grid.extraction_area_tl[0]/(W-1), self.grid.extraction_area_tl[1]/(H-1),
        self.grid.extraction_area_br[0]/(W-1), self.grid.extraction_area_br[1]/(H-1),

        #Show Progress
        cheb_dist((ax, ay), (gx, gy)) / max(W-1, H-1),
        self.surroundCounter / numAgents,
        self.holdCounter / self.timeToKill
    ]

    return obs


#State

def get_state(self):

    W, H = self.gridwidth, self.gridheight
    s = []

    # all agents
    for a in self.engine.agents:
        s += [a.x/(W-1), a.y/(H-1)]

    #Ghost + visible flag (full obs baseline → 1.0)
    gx, gy = self.engine.ghostCoords
    s += [gx/(W-1), gy/(H-1), 1.0]

    #Extraction area
    etlx, etly = self.engine.grid.extraction_area_tl
    ebrx, ebry = self.engine.grid.extraction_area_br
    s += [etlx/(W-1), etly/(H-1), ebrx/(W-1), ebry/(H-1)]

    #Game progress
    s += [self.engine.Time / self.episode_limit]

    return np.asarray(s, dtype=np.float32)

In [ ]:
#Rewards


    def globalReward(self, prev_d_ghost_extract, prev_d_ghost_agents, prev_surround_count):
        reward = -0.01 
        terminated = False
        success = False

        if self.isSurrounded():
            self._hold_counter += 1
        if self.surroundCounter >= numAgents:
            self.holdCounter += 1
        else:
            self._hold_counter = 0
            self.holdCounter = 0

        terminated = False
     
        # End conditions
        if self.holdCounter >= self.timeToKill:
            reward += self.win_reward  
            terminated = True
            success = True
        elif self.Time >= self.episodeLimit:
            terminated = True
            reward -= 1000.0  

        if terminated:
            return reward, terminated, success
        
        #Reward for surrounding ghost
        reward += self.Surround_Reward(prev_surround_count)


        return reward, terminated, success


def AgentClumpingPenalty(self):
        penalty = 0.0
        CLUMP_DISTANCE = 2
        
        # Only penalize if not currently surrounding ghost
        if self.surroundCounter >= numAgents - 1:
            return 0.0  # Allow clustering when capturing
        
        for i in range(len(self.agents)):
            for j in range(i + 1, len(self.agents)):
                dist = cheb_dist((self.agents[i].x, self.agents[i].y), 
                                 (self.agents[j].x, self.agents[j].y))
               
                if dist <= CLUMP_DISTANCE:
                    penalty += 0.1  
        
        return penalty


 def Surround_Reward(self, prev_surround_count):
        reward = 0.0
        new_surround_count = self.ghost.GetSurroundedCount(self.agentCoords)

        # Progressive surround rewards - each agent joining is valuable
        if new_surround_count > prev_surround_count:
            # Increasing reward per agent (more agents = more valuable)
            reward += self.reward_new_surround * (new_surround_count - prev_surround_count)


        # Bonus for achieving full surround (all agents within radius)
        if new_surround_count >= numAgents:
            reward += self.reward_surrounded

        return reward

<h2> Stage 1 </h2>
Hold time: 3 step
No extraction
No ghost movement

<h2> VDN </h2>
Did not work. This was the last and best attempt
<pre>
[INFO 10:16:01] my_main Recent Stats | t_env:     199894 | Episode:      676
ep_length_mean:          300.0000       epsilon:                   0.0500       grad_norm:               2968.1982      hold_mean:                 0.0000
loss:                    9660.9391      q_taken_mean:              4.3232       return_mean:             462.8749       return_std:              980.8482
success_mean:              0.0000       t_mean:                  300.0000       target_mean:               5.4364       td_error_abs:             10.1837
test_ep_length_mean:     286.3500       test_hold_mean:            0.2500       test_return_mean:        841.0543       test_return_std:         1298.6543
test_success_mean:         0.0500       test_t_mean:             286.3500
[INFO 10:16:01] my_main Finished Training
</pre>

In [20]:
Image(url="ResultImages/Baseline_2_BeforeQMIX.png", width=900, height=600)

VDN Seems to not be able to model the collaboration needed for this task

<h2> QMIX</h2>
Second attempt at baseline

First Attempt:

<pre> 

ep_length_mean:          296.0036       epsilon:                   0.0500       grad_norm:               44408.9617     hold_mean:                 0.2679
loss:                    15895.5070     q_taken_mean:              8.6338       return_mean:             1068.4453      return_std:              1801.3856
success_mean:              0.0536       t_mean:                  296.0036       target_mean:               8.5500       td_error_abs:             21.6396
test_ep_length_mean:     262.0500       test_hold_mean:            0.7500       test_return_mean:        2047.0497      test_return_std:         2918.8472
test_success_mean:         0.1500       test_t_mean:             262.0500

</pre>

In [19]:
# import image module
from IPython.display import Image

# get the image
Image(url="ResultImages/BaselineFirstQMIX.png", width=900, height=600)

I notice that gradient is EXPLODING and loss is huge. I change my reward structure to make it smoother and more normalized

Second Attempt:
ep_length_mean:          250.6667       epsilon:                   0.0500       grad_norm:                 1.3428       hold_mean:                 1.2222
loss:                      0.7356       q_taken_mean:              0.2904       return_mean:             -248.2167      return_std:              520.8183
success_mean:              0.2444       t_mean:                  250.6667       target_mean:               0.2837       td_error_abs:              0.1454
test_ep_length_mean:     271.5500       test_hold_mean:            0.7500       test_return_mean:        -160.0374      test_return_std:         573.6516

In [21]:
Image(url="ResultImages/QMIX2ndBaseline.png", width=900, height=600)

Clearly doing much better and trending in correct direction. 
We see that QMIX is doing much better, and a better reward structure is paying off. 

We are going in the right direction, now that we see some structure and learning, let's tweak our reward structure

In [ ]:
#Make distance shaping even better : exponential pull towards ghost   

def AgentToGhostDist_Reward(self, prev_agent_ghost_d):
    reward = 0.0

    # Closer to ghost = higher reward every step
    max_dist = max(self.grid.width, self.grid.height) - 1
    
    for a in self.agents:
        dist = cheb_dist((a.x, a.y), self.ghostCoords)
        norm_dist = dist / max_dist
        reward += self.lambda_agent_dist_to_ghost * ((1.0 - norm_dist) ** 2) * 2.0 

    return reward

We still get some Q exploitation behaviour, so I add progress markers into state so Q functions can better estimate value

In [ ]:
    def get_state(self):
        """
        Previous stuff
        """

        #Global task progress (for value estimation)
        s += [
            self.engine.surroundCounter / self.n_agents,  # surround progress [0-1]
            # Average distance to ghost (normalized)
            sum(cheb_dist((a.x, a.y), self.engine.ghostCoords) for a in self.engine.agents) / (self.n_agents * max(W-1, H-1))
        ]

        return np.asarray(s, dtype=np.float32)

<h2> BOOM here we are </h2>

<pre>

[INFO 11:05:27] my_main Recent Stats | t_env:     140057 | Episode:     1484
ep_length_mean:           30.6744       epsilon:                   0.3431       grad_norm:                31.0257       hold_mean:                 5.0000
loss:                      1.9014       q_taken_mean:              2.5236       return_mean:             320.9782       return_std:               71.2775
success_mean:              1.0000       t_mean:                   30.6744       target_mean:               2.4942       td_error_abs:              0.7269
test_ep_length_mean:      32.1500       test_hold_mean:            5.0000       test_return_mean:        365.2984       test_return_std:         108.3838
test_success_mean:         1.0000       test_t_mean:              32.1500
</pre>

Exponential shaping made a huuuuuuge difference

In [22]:
Image(url="ResultImages/WorkingQMIXBaseline.png", width=900, height=600)

<h3> Wait: Maybe it wasn't VDN's fault? </h3>

Proof that VDN could not do this: 
Same env, rewards, etc, just with vdn

<pre>

[INFO 11:09:27] my_main Recent Stats | t_env:     198294 | Episode:      672
ep_length_mean:          295.8000       epsilon:                   0.0500       grad_norm:               112.0349       hold_mean:                 0.1250
loss:                     10.1877       q_taken_mean:              3.3301       return_mean:             762.8890       return_std:              271.7277
success_mean:              0.0250       t_mean:                  295.8000       target_mean:               3.3562       td_error_abs:              1.4568
test_ep_length_mean:     300.0000       test_hold_mean:            0.0000       test_return_mean:        897.3207       test_return_std:         192.5651
test_success_mean:         0.0000       test_t_mean:             300.0000

</pre>

<h1>Stage 2: Capturing a moving ghost</h1>
Great we can capture a static ghost. Now let's make the ghost move and make this task much harder

Ghost now moves 30% of the time, with a 20% decrease for every agent near it
Spawns further away : 15

First attempt gives 35% success rate. Its ok but not great. We want this to improve <br>
We test the policy and notice that they quickly get to it and close to it, but can struggle to fully surround and capture.


In [24]:
from IPython.display import Video

Video("QMIX_FullObv_Stage2_CantCatch.mov", embed=True)

I notice ghosts tend to enter surround and leave again and again, so I will strongly reward staying in surround radius, and decay the bonus for a new complete surround.
Also will encourage coverage on multiple directions/quadrants of the ghost

In [ ]:
def globalReward(self, prev_d_ghost_extract, prev_d_ghost_agents, prev_surround_count):
        reward = -0.01  
        terminated = False
        success = False

        if self.surroundCounter >= numAgents:
            self.holdCounter += 1
        else:
            self.holdCounter = 0

        # End conditions
        if self.holdCounter >= self.timeToKill:
            reward += self.win_reward  # HUGE success reward to make it unmissable
            terminated = True
            success = True
        elif self.Time >= self.episodeLimit:
            terminated = True


        if terminated:
            return reward, terminated, success
        
        # Reward for getting agents closer to ghost
        reward += self.AgentToGhostDist_Reward(prev_d_ghost_agents)

        #Reward for surrounding ghost
        reward += self.Surround_Reward(prev_surround_count)

        if self.surroundCounter >= 2:  # Only when at least 2 agents nearby
            reward += self.QuadrantCoverage_Reward()

        if self.surroundCounter < numAgents:  #when NOT surrounding ghost)
            reward += self.AgentClumpingPenalty()

        return reward, terminated, success


def QuadrantCoverage_Reward(self):
        
        gx, gy = self.ghostCoords
        directions = [False, False, False, False]  # Left, Right, Above, Below
        
        for ax, ay in self.agentCoords:
            # Only count agents within surround radius
            if cheb_dist((ax, ay), (gx, gy)) <= self.surroundRadius:
                dx = ax - gx
                dy = ay - gy
                
                # Determine primary direction (using dominance of dx vs dy)
                if abs(dx) > abs(dy):  # Horizontal dominance
                    if dx > 0:
                        directions[1] = True  # Right
                    else:
                        directions[0] = True  # Left
                else:  # Vertical dominance
                    if dy > 0:
                        directions[2] = True  # Above
                    else:
                        directions[3] = True  # Below
        
        # Reward per direction covered
        num_covered = sum(directions)
        return self.lambda_quadrant_coverage * num_covered

From visual testing what is happening is that they get close very quickly, but struggle to fully surround and capture. It seems like agents get in the way, and possibly exploit coming in and out of surround radius

<h3> Weird Training Behaviour</h3>

Results great until 200k steps, then things break. Seems like issues with epsilon: exploration etc.
So the reward structure is working well, but now likely issues with learning algo details

In [31]:
Image(url="ResultImages/QMIX_Stage2_BrokenAt200k.png", width=900, height=600)

At this point I decided to try stage 3 and basically same rewards and same behaviour. 

So I'm going to try enabling adding agent id to observations to allow for more heterogenous actions and more "separation" between the agents. 
I Cubed the distance so they wanna get even closer
Then I made the avoid radius smaller

<h1> BIG Bug found: Incorrect order of rewards </h1>
NOTHING was working. So I had to dig deeper to analyze and given their erratic behaviour.
Was suspucious of an underlying problem so I looked at step how movement was working. This lead me to step(). 

In [ ]:
#Before
# 1. Agents act
for a, act_id in zip(self.agents, actions):
    a.apply_action(act_id)

# 2. Ghost moves (AFTER agents)
self.ghostCoords = self.ghost.move(self.agentCoords)

# 3. Calculate rewards based on NEW positions
reward = self.globalReward(...)

In [ ]:
#After
self.ghostCoords = self.ghost.move(self.agentCoords)

# 2. Agents act (seeing where ghost went)
for a, act_id in zip(self.agents, actions):
    a.apply_action(act_id)

# 3. Calculate rewards
reward = self.globalReward(...)

Before, we had action -> state change -> reward

What we SHOULD have is state change -> act accordingly -> reward
So we had a small but very important missmatch on what we were doing vs getting rewarded for

Why did we not notice this earlier? The Ghost wasn't moving, so it didnt update the state. Now only that the ghost moves and we depend on it's movement, did we notice


In [29]:
Image(url="ResultImages/QMIX_Stage3_UpdatedStep.png", width=900, height=600)

Amazing. Now for some final reward shaping additions:

In [ ]:
def AgentToGhostDist_Reward(self, prev_agent_ghost_d):
        reward = 0.0

        max_dist = max(self.grid.width - 1, self.grid.height - 1)
        
        for a in self.agents:
            dist = cheb_dist((a.x, a.y), self.ghostCoords)
            norm_dist = dist / max_dist
            # STRONGER exponential: cube instead of square for aggressive pull when close
            reward += self.lambda_agent_dist_to_ghost * ((1.0 - norm_dist) ** 3) * 3.0

        return reward
    
def Surround_Reward(self, prev_surround_count):
        reward = 0.0
        new_surround_count = self.ghost.GetSurroundedCount(self.agentCoords)

        # Progressive surround rewards - BIG bonus for entering radius
        if new_surround_count > prev_surround_count:
            reward += self.reward_new_surround * (new_surround_count - prev_surround_count) * 2.0
        
        # Penalty Breaking formation
        if new_surround_count < prev_surround_count:
            penalty = self.penalty_formation_break * (prev_surround_count - new_surround_count) * 0.5
            reward -= penalty
        
        
        # Keeping full surround
        if new_surround_count >= numAgents:
            reward += self.reward_full_surround

        return reward

In [32]:
Image(url="ResultImages/WorkingQMIXBaseline.png", width=900, height=600)

<h1> Stage 3: Extraction </h1>
We move closer to our end goal of catching -> move to extraction sites

First attempts fail. 

Not going well. 


In [ ]:

#For shaping rewards per phase of game
surround_progress = self.surroundCounter / numAgents 

extraction_multiplier = 0.1 + 1.9 * (surround_progress ** 3)
surrounding_multiplier = 1.0

#STEP1: Surrounding
reward += surrounding_multiplier * self.AgentToGhostDist_Reward(prev_d_ghost_agents)
reward += surrounding_multiplier * self.Surround_Reward(prev_surround_count)
reward += surrounding_multiplier * self.QuadrantCoverage_Reward()

#Step2: Extraction
reward += extraction_multiplier * self.GhostToExtraction_Reward(prev_d_ghost_extract)

return reward, terminated, success

Now I will heavily reward not (only) surrounding, but leaving a hole aka pushing on the correct side. 

In [30]:
def DirectionalPush_Reward(self):
    #Reward agents for positioning to PUSH the ghost toward extraction.

    gx, gy = self.ghostCoords
    ex, ey = self.grid.extraction_point_center
    
    # Vector from ghost to extraction (desired direction for ghost to move)
    ghost_to_extract_x = ex - gx
    ghost_to_extract_y = ey - gy
    extract_dist = max(abs(ghost_to_extract_x), abs(ghost_to_extract_y))  # Chebyshev distance
    
    # Avoid division by zero if ghost is at extraction
    if extract_dist < 0.1:
        return 0.0
    
    # Normalize the extraction direction vector
    extract_dir_x = ghost_to_extract_x / extract_dist
    extract_dir_y = ghost_to_extract_y / extract_dist
    
    total_reward = 0.0
    max_dist = max(self.grid.width, self.grid.height) - 1
    
    for ax, ay in self.agentCoords:
        ghost_to_agent_x = ax - gx
        ghost_to_agent_y = ay - gy
        agent_dist = cheb_dist((ax, ay), (gx, gy))
        
        
        # Normalize agent direction vector
        agent_dir_x = ghost_to_agent_x / max(agent_dist, 1)
        agent_dir_y = ghost_to_agent_y / max(agent_dist, 1)
        
        # Dot product: want agents opposite extraction direction, so we want negative
        dot_product = (agent_dir_x * extract_dir_x + agent_dir_y * extract_dir_y)
        
        # Calculate proximity bonus. Reward when close
        normalized_dist = agent_dist / max_dist
        proximity_multiplier = max(0, (1.0 - normalized_dist)) ** 1.5 
        
        # Simple exponential reward: maximize alignment opposite to extraction direction
        alignment_score = (-dot_product + 1) / 2  # Maps [-1, 1] to [1.0, 0.0]
        directional_reward = self.lambda_quadrant_coverage * proximity_multiplier * (alignment_score ** 3)
        total_reward += directional_reward  

    return total_reward

<h2> EXTRACTION DID NOT WORK </h2>

Accept defeat, reframe final goal. We will just be catching under partial observability.

<h1> Catching Only: Partial Observability </h1>

In [ ]:
#New Observations

def getAgentObs(self, agentIndex):
    W, H = self.grid.width, self.grid.height
    ax, ay = self.agents[agentIndex].x, self.agents[agentIndex].y
    
    obs = [ax / (W-1), ay / (H-1)]
    
    if self.ghost_visible:
        gx, gy = self.ghostCoords
        obs += [gx / (W-1), gy / (H-1), 1.0]
        dist_to_ghost = cheb_dist((ax, ay), (gx, gy)) / max(W-1, H-1)
    else:
        obs += [0.0, 0.0, 0.0]
        dist_to_ghost = 0.0
    
    visible_agents = []
    for other_idx in range(len(self.agents)):
        if other_idx == agentIndex:
            continue
        if self.canSeeAgent(agentIndex, other_idx):
            ox, oy = self.agents[other_idx].x, self.agents[other_idx].y
            dist = cheb_dist((ax, ay), (ox, oy))
            visible_agents.append((dist, ox, oy))
    
    visible_agents.sort(key=lambda x: x[0]) #Sort agent visibility by closest first
    
    for i in range(len(self.agents) - 1):
        if i < len(visible_agents):
            dist, ox, oy = visible_agents[i]
            obs += [ox / (W-1), oy / (H-1), 1.0]
        else:
            obs += [0.0, 0.0, 0.0]
    
    obs += [
        self.grid.extraction_area_tl[0]/(W-1), 
        self.grid.extraction_area_tl[1]/(H-1),
        self.grid.extraction_area_br[0]/(W-1), 
        self.grid.extraction_area_br[1]/(H-1),
        dist_to_ghost
    ]

    return obs




#New State


def get_state(self):
    W, H = self.gridwidth, self.gridheight
    s = []

    # all agents
    for a in self.engine.agents:
        s += [a.x/(W-1), a.y/(H-1)]

    if self.engine.ghost_visible:
        gx, gy = self.engine.ghostCoords
        s += [gx/(W-1), gy/(H-1), 1.0]
    else:
        s += [0.0, 0.0, 0.0]

    #Extraction area
    etlx, etly = self.engine.grid.extraction_area_tl
    ebrx, ebry = self.engine.grid.extraction_area_br
    s += [etlx/(W-1), etly/(H-1), ebrx/(W-1), ebry/(H-1)]

    #Game progress
    s += [self.engine.Time / self.episode_limit]

    #Global task progress (for value estimation)
    s += [
        self.engine.surroundCounter / self.n_agents,
        self.engine.holdCounter / self.engine.timeToKill,
    ]
    
    #Only include avg distance to ghost if visible
    if self.engine.ghost_visible:
        avg_dist = sum(cheb_dist((a.x, a.y), self.engine.ghostCoords) for a in self.engine.agents) / (self.n_agents * max(W-1, H-1))
        s += [avg_dist]
    else:
        s += [0.0]

    return np.asarray(s, dtype=np.float32)

First attempt somme success towards end of training. I notice they spam in out of spotting it, so I will add decay to this reward. 
Given towards end of training, I will increase training time, and update epsilon for more exploration

Adding reward for spread coverage when agent is not visible

Curriculum Based: we start with large vision radius and ghost less mobile

<h2> Stage 1 </h2>

  vision_radius: 10
  spawn_radius: 11
  ghost_move_prob: 0.5

Statistically going pretty well. From video performance we see that once they see the ghost, they catch it very quickly, but they don't seem to be "looking" for it and rather bump into it pretty much at random

In [33]:
Image(url="ResultImages/PartObvStage1_First.png", width=900, height=600)

We will shape rewards for exploration, and reduce vision radius. This will force them into exploration

Video Name: FullObv_Stage1_FirstSuccess

Catching is working great, but it seems like they aren't doing much searching/exploration

<h2> Stage 2: Smaller vision, more mobile ghost </h2>

We see that we can catch well, now let's incentivise searching

In [ ]:
def GridCoverageReward(self):
    exploration_reward = 0.0
    
    for agent in self.agents:
        cell = (agent.x, agent.y)
        visit_count = self.visit_map.get(cell, 0)
        
        # Reward inversely proportional to visit count
        cell_reward = 1.0 / (1.0 + visit_count)
        exploration_reward += self.lambda_grid_coverage * cell_reward
        
        # Update visit count
        self.visit_map[cell] = visit_count + 1
    
    return exploration_reward



##Global Reward Function 

...

    ####Finding Ghost
    if not prev_ghost_visible and self.ghost_visible:
        reward += self.reward_ghost_spotted
        self.reward_ghost_spotted *= 0.8  # Decay for future sightings

    if self.ghost_visible:
        reward += self.reward_ghost_visible

    if not self.ghost_visible:
        reward += self.GridCoverageReward()
        reward += self.AgentSpreadReward()

    ####Catching Ghost
    if self.ghost_visible:
        reward += self.AgentToGhostDist_Reward(prev_d_ghost_agents)
        reward += self.Surround_Reward(prev_surround_count)
        reward += self.QuadrantCoverage_Reward()  

    return reward, terminated, success

<h3> New from scratch policy </h3>

Video: PartObvStage2_Quite well
Catch very well. Some issues with exploration, but not terrible

In [36]:
Image(url="ResultImages/PartObvStage2_FirstAttempt.png", width=900, height=600)

Adding more incentive for exploration: only rewarding whewn they move. This should help the spazzy behaviour

Amazing results, statistically and video wise

Video: 

In [38]:
Image(url="ResultImages/PartObv_Stage2_Winner.png", width=900, height=600)

NEXT STEPS: Even more difficult. Decrease vision radius, make ghost avoid earlier
This policy was trained on top of stage 2

In [39]:
Image(url="ResultImages/PartObv_Stage3.png", width=900, height=600)

TODO Video

We've clearly taught these agents to look for the ghost, and then to catch it once they do. Despite all the training, this is still a difficult task.